# Defining a graph structure

Graph class definition

In [44]:
m = {i : False for i in V}
Q = [1]
while Q != [] :
    i = Q.pop( 0 )
    m[i] = True
    L = [ j for (k,j) in inc[i] if k == i ] + [ j for (j,k) in incidentes(i) if k == i ]
    Q += [ i for i in L if ( not m[i] ) and ( i not in Q )]

In [72]:
def deep_first_search( i, inc, m ) :
    m[i] = True
    N = [ j for (k,j) in inc[i] if k == i ] + [ j for (j,k) in inc[i] if k == i ]
    print( i, N )
    for j in N :
        if not m[j] :
            dfs( j, inc, m )

m = { i : False for i in V }
dfs( 1, inc, m )

1 [2, 3, 4]
2 [6, 1]
6 [2]
3 [5, 1]
5 [3, 4]
4 [5, 1]


# Kruskal's algorithm

In [98]:
cc = {E[i] : c[i] for i in range( len( E ) ) }
print( 'cc = {}'.format( cc ) )

cc = {(1, 2): -5, (1, 3): 4, (2, 3): 3, (2, 4): -7, (3, 4): 5, (4, 1): 1, (5, 6): 0, (3, 5): 2, (4, 6): 5}


In [92]:
E.sort( key = lambda x : cc[ x ] )

In [100]:
T = []
S = { i : i for i in V }
for ( i, j ) in E :
    if ( S[i] == S[j] ) :
        continue
    T.append( ( i, j ) )
    s_i = S[i]
    for k in V :
        if S[k] == s_i :
            S[k] = S[j]

print( T )

[(1, 2), (1, 3), (2, 4), (5, 6), (3, 5)]


In [102]:
LS = {i : [i] for i in V}
print( LS )

{1: [1], 2: [2], 3: [3], 4: [4], 5: [5], 6: [6]}
